In [2]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv
from bson.decimal128 import Decimal128

# ---------------------------
# Config & Connect
# ---------------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# ---------------------------
# Your payload
# ---------------------------
payload = {
    "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
    "current_form_qr_code": "20251013",
    "previous_form_qr_code": "20251012",
    "tin": "123-456-789",
    "location": "JEF Gas Station – Sikatuna Branch",
    "date": "2025-10-13",
    "created": "2025-10-13T05:03:46.557977Z",
    "cashier_employee_number": "11111",
    "recorder_employee_number": "22222",
    "items": [
        {
            "receipt_number": "123456",
            "customer_name": "Markus",
            "type": "fuel",
            "vatable_sales": "50.00",
            "vat_amount": "6.00",
            "total_amount": "56.00"
        },
        {
            "receipt_number": "444587",
            "customer_name": "Mike",
            "type": "fuel",
            "vatable_sales": "120.00",
            "vat_amount": "14.40",
            "total_amount": "134.40"
        }
    ]
}

# ---------------------------
# Helpers
# ---------------------------
def _to_decimal128(x):
    if isinstance(x, Decimal128):
        return x
    # Accept str/int/float; use string path to preserve exact value
    return Decimal128(str(x))

def _normalize_payload(doc):
    doc = dict(doc)  # shallow copy
    items = []
    for it in doc.get("items", []):
        it = dict(it)
        for k in ("vatable_sales", "vat_amount", "total_amount"):
            if k in it and it[k] is not None:
                it[k] = _to_decimal128(it[k])
        items.append(it)
    doc["items"] = items
    return doc

norm_payload = _normalize_payload(payload)

# ---------------------------
# Find latest EMPTY form
# ---------------------------
latest_empty = col.find_one(
    {
        "$or": [
            {"items": {"$exists": False}},
            {"items": {"$size": 0}}
        ]
    },
    sort=[("created", -1), ("_id", -1)]
)

if not latest_empty:
    print("❌ No EMPTY sales_invoices form found. Nothing to patch.")
else:
    result = col.update_one(
        {"_id": latest_empty["_id"]},
        {
            "$set": {
                "id": norm_payload["id"],
                "current_form_qr_code": norm_payload["current_form_qr_code"],
                "previous_form_qr_code": norm_payload["previous_form_qr_code"],
                "tin": norm_payload["tin"],
                "location": norm_payload["location"],
                "date": norm_payload["date"],
                "created": norm_payload["created"],
                "cashier_employee_number": norm_payload["cashier_employee_number"],
                "recorder_employee_number": norm_payload["recorder_employee_number"],
                "items": norm_payload["items"],
            }
        }
    )
    updated = col.find_one({"_id": latest_empty["_id"]})
    print("✅ Patched existing EMPTY form.")
    print(json.dumps(updated, indent=2, default=str))


❌ No EMPTY sales_invoices form found. Nothing to patch.
